Extract Snotel location data from Snobal run results

In [ ]:
import datetime
import xarray as xr

from pathlib import PurePath, Path

from dask_cluster import CHPC, start_cluster
from snote_sites import SNOTEL

In [ ]:
client = start_cluster(4, 8)
client

In [ ]:
home = PurePath(f'{Path.home()}/scratch/iSnobal/runs/run201[7,8]*')
snow_nc = 'snow.nc'

In [ ]:
snow = xr.open_mfdataset(
    str(home.joinpath(snow_nc)),
    parallel=True,
).sel(time=datetime.time(23))

In [ ]:
comp = dict(zlib=True, complevel=5)

def write_snotel(site):
    snotel_site = snow.sel(x=SNOTEL[site]['snobal']['lon'], y=SNOTEL[site]['snobal']['lat'])
    encoding = {var: comp for var in snotel_site.data_vars}

    snotel_site.to_netcdf(
        PurePath(f'{Path.home()}/scratch/iSnobal/{site}-snobal.zarr').as_posix(), 
        encoding=encoding,
        compute=True
    )

In [ ]:
for site in SNOTEL.keys():
    write_snotel(site)